In [13]:
import os

In [14]:
%pwd

'd:\\GitHub_Repos\\Chicken-Disease-Classification-MLOps-DVC'

In [21]:
os.chdir("d:\\GitHub_Repos\\Chicken-Disease-Classification-MLOps-DVC")

In [22]:
%pwd

'd:\\GitHub_Repos\\Chicken-Disease-Classification-MLOps-DVC'

In [23]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [24]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [25]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [34]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [35]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        try:
            if not os.path.exists(self.config.local_data_file):
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                logger.info(f"{filename} downloaded! With following info: \n{headers}")
            else:
                logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
        except Exception as e:
            logger.error(f"Error occurred during file download: {e}")
            raise


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [36]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"An error occurred: {e}")
    raise

[2024-12-25 19:26:12,790: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-25 19:26:12,792: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-25 19:26:12,794: INFO: common: created directory at : artifacts]
[2024-12-25 19:26:12,796: INFO: common: created directory at : artifacts/data_ingestion]
[2024-12-25 19:26:14,413: INFO: 2096091580: artifacts/data_ingestion/data.zip downloaded! With following info: 
Connection: close
Content-Length: 12434258
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "49270f7cc136a0a55d089348ad8e86855f5fceb53359fffa350d9c1166162a7b"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: EA4A:23F678:978CF0:A7BD9F:676CA2A5
Accept-Ranges: bytes
Date: Thu, 26 Dec 2024 00:26:13 GMT
Via: 1.1 varnish
X-Served-By: cache-ewr-kewr1740046-EW